In [1]:
import os.path
import tensorflow as tf
import warnings
import numpy
import scipy
import helper
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    image_input = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = tf.get_default_graph().get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = tf.get_default_graph().get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = tf.get_default_graph().get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)
print('load_vgg passed')

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    init = tf.truncated_normal_initializer(stddev = 0.01)
    regu = tf.contrib.layers.l2_regularizer(1e-3)
    def conv_1x1(x, num_classes, init = init):
        return tf.layers.conv2d(x, num_classes, 1, padding = 'same', kernel_initializer = init, kernel_regularizer = regu)

    def upsample(x, num_classes, depth, strides, init = init):
        return tf.layers.conv2d_transpose(x, num_classes, depth, strides, padding = 'same', kernel_initializer = init, kernel_regularizer = regu)
    
    layer_7_1x1 = conv_1x1(vgg_layer7_out, num_classes)
    layer_4_1x1 = conv_1x1(vgg_layer4_out, num_classes)
    layer_3_1x1 = conv_1x1(vgg_layer3_out, num_classes)

    #implement the first transposed convolution layer
    upsample1 = upsample(layer_7_1x1, num_classes, 5, 2)
    layer1 = tf.layers.batch_normalization(upsample1)
    #add the first skip connection from the layer_4_1x1    
    layer1 = tf.add(layer1, layer_4_1x1)

    #implement the another transposed convolution layer
    upsample2 = upsample(layer1, num_classes, 5, 2)
    layer2 = tf.layers.batch_normalization(upsample2)
    #add the second skip connection from the layer_3_1x1        
    layer2 = tf.add(layer2, layer_3_1x1)

    return upsample(layer2, num_classes, 14, 8)    

tests.test_layers(layers)
print("test_layers passed")

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1,num_classes))
    # define loss function
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= correct_label))
    # define training operation
    optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss
tests.test_optimize(optimize)
print('test optimization passed')

def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    sess.run(tf.global_variables_initializer())
    
    print("Training...")
    print()
    for epoch in range(epochs):
        print("EPOCH {} ...".format(epoch+1))
        
        kr = 0.5
        initial_lr = 0.0001
        
        if epoch > 5 and epoch <= 15:
            lr = initial_lr / 5
        elif epoch > 15 and epoch <= 25:
            lr = initial_lr / 25
        elif epoch > 25:
            lr = initial_lr / 100
        else:
            lr = initial_lr

        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], 
                               feed_dict={input_image: image, correct_label: label, keep_prob: kr, learning_rate: initial_lr})
            print("Loss: = {:.3f}".format(loss))
        print()
tests.test_train_nn(train_nn)
print('test_train_nn passed')

def run():
    num_classes = 2
    epochs = 10
    batch_size = 5

    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    #  no way!
    
    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        vgg_input, keep_prob, vgg_layer3, vgg_layer4, vgg_layer7 = load_vgg(
            sess, vgg_path)

        nn_last_layer = layers(vgg_layer3, vgg_layer4, vgg_layer7, num_classes)

        correct_label = tf.placeholder(tf.float32, [None, image_shape[0], image_shape[1], num_classes])
        learning_rate = tf.placeholder(tf.float32)

        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)
        sess.run(tf.global_variables_initializer())        

        # Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, vgg_input,
             correct_label, keep_prob, learning_rate)

        # Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, vgg_input)

        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()


TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0
Tests Passed
load_vgg passed
Tests Passed
test_layers passed
Tests Passed
test optimization passed
Tests Passed
test_train_nn passed
Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Training...

EPOCH 1 ...
Loss: = 1.148
Loss: = 1.182
Loss: = 0.967
Loss: = 0.898
Loss: = 0.835
Loss: = 0.825
Loss: = 0.822
Loss: = 0.757
Loss: = 0.748
Loss: = 0.736
Loss: = 0.734
Loss: = 0.730
Loss: = 0.723
Loss: = 0.718
Loss: = 0.708
Loss: = 0.703
Loss: = 0.700
Loss: = 0.695
Loss: = 0.694
Loss: = 0.693
Loss: = 0.688
Loss: = 0.685
Loss: = 0.679
Loss: = 0.675
Loss: = 0.671
Loss: = 0.666
Loss: = 0.658
Loss: = 0.649
Loss: = 0.650
Loss: = 0.639
Loss: = 0.617
Loss: = 0.616
Loss: = 0.628
Loss: = 0.618
Loss: = 0.590
Loss: = 0.579
Loss: = 0.577
Loss: = 0.592
Loss: = 0.541
Loss: = 0.544
Loss: = 0.545
Loss: = 0.496
Loss: = 0.494
Loss: = 0.506
Loss: = 0.492
Loss: = 0.461
Loss: = 0.437
Loss: = 0.470
Loss: = 0.434
Loss: = 0.383


Loss: = 0.056
Loss: = 0.048
Loss: = 0.070
Loss: = 0.073
Loss: = 0.062
Loss: = 0.058
Loss: = 0.046
Loss: = 0.113
Loss: = 0.093
Loss: = 0.055
Loss: = 0.066
Loss: = 0.087
Loss: = 0.115
Loss: = 0.068
Loss: = 0.052
Loss: = 0.083
Loss: = 0.041
Loss: = 0.070
Loss: = 0.047
Loss: = 0.043
Loss: = 0.069
Loss: = 0.080
Loss: = 0.068
Loss: = 0.067

Training Finished. Saving test images to: ./runs/1507633227.9770951
